<a href="https://colab.research.google.com/github/afcabre/git-25-09-gh/blob/main/ValoracionEmpresas_USA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO FINAL: Valoración de Fundamentales de Empresas Estadounidenses que cotizan en Bolsa y Agente SQL con LangChain

**Autor:** Andrés Fernando Cabrera - Curso de Fundamentos de LLM y Datos  
**Sesión:** Preprocesamiento de Datos y Agentes SQL

---
## 1. Introducción: de Estados Financieros y Precios a Inteligencia de Inversión Conversacional

El escenario propuesto se plantea bajo el contexto de análisis de inversiones, y la resolución de las preguntas recurrentes que se suelen enfrentar antes de hacer o liquidar una inversión, por ejemplo, se quiere saber si el precio que se está pagando o recibiendo es considerado justo, es económico o costos, frente a sus pares. O se necesita identificar señales de riesgo financiero antes de comprar, y se debe establecer de manera clara, con argumentos provenientes del análisis fundamental, por qué una acción luce atractiva o costosa en un momento específico.
Tradicionalmente, responder estas preguntas implica construir queries SQL distintas, cruzar estados financieros con precios, calcular métricas derivadas (TTM, márgenes, yields, endeudamiento) y luego consolidar hallazgos en reportes. Este flujo es lento, repetitivo y difícil de escalar cuando las preguntas se multiplican.

Este proyecto final transforma ese proceso manual en un sistema de **análisis conversacional** soportado por un **agente (LLM) conectado a una base de datos SQL**. El pipeline toma datos públicos de SimFin (estados financieros trimestrales y precios diarios de empresas de USA), los procesa y estructura en un esquema relacional, y habilita un agente capaz de responder preguntas en lenguaje natural. El resultado esperado es una interfaz donde un usuario sin conocimiento de SQL puede explorar, filtrar y explicar oportunidades basadas en fundamentales, con trazabilidad hacia las columnas fuente del dataset.

### Objetivos de aprendizaje (enfoque de apropiación)

Se busca demostrar apropiación de lo visto en arquitecturas de agentes con SQL, mediante decisiones de diseño y pruebas que conectan datos con preguntas reales de análisis financiero:

**Diseño de esquema relacional pensando en el agente:**  
Se busca diseñar tablas y relaciones que faciliten el razonamiento: dimensiones (empresas, industrias), hechos (balance, income, cashflow, precios), y una capa de métricas derivadas con trazabilidad. El objetivo no es solo normalizar, sino habilitar consultas repetibles y comprensibles para un asistente conversacional.

**Orquestación del agente para preguntas en lenguaje natural:**  
EL agente SQL que no solo traduce la preguntas a queries, sino quedebe mantener contexto y usar el lenguaje del dominio. Por ejemplo: “barata vs su industria” implica comparar percentiles sectoriales; “señales de riesgo” implica revisar deuda, liquidez y cobertura; “mejora sostenida” implica tendencias y estabilidad, no un trimestre aislado. Estas capacidades se prueban con un set de preguntas guía y casos de prueba.

Al finalizar, el objetivo es contar con un prototipo funcional y, sobre todo, con un entendimiento práctico de cómo **estructurar datos y métricas para maximizar su utilidad en aplicaciones de IA conversacional** apoyadas en SQL.

---

# 2. Exploración inicial de datos

## 2.1 Instalación de Dependencias
Instalar librerías base para ingestión de CSV, SQL (SQLite) y agente conversacional (LangChain + OpenAI).

In [3]:
!pip -q install -U \
  "pandas==2.2.2" \
  "numpy==2.0.2" \
  pyarrow sqlalchemy tabulate \
  langchain langchain-openai langchain-community \
  openai tiktoken

## 2.2 Importación de librerías
Cargar librerías de trabajo (pandas/numpy para DataFrames, pathlib para rutas, IPython para visualización).

In [2]:
import os
import io
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
from pathlib import Path
from IPython.display import display
from datetime import datetime

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

## 2.3 Carga del Dataset Original

Descomprimir us-shareprices-daily.zip en /content/data para obtener us-shareprices-daily.csv. Cargar los 6 CSV (separador ;) desde /content/data en DataFrames y confirmar dimensiones por archivo y mostrar las primeras filas de cada DataFrame para validar que la carga fue correcta.

In [4]:
# 2.3 Carga del Dataset Original (con soporte ZIP para prices)

from pathlib import Path
import zipfile
import pandas as pd

DATA_DIR = Path("/content/data")

# --- 2.3.1: Descomprimir prices si viene como ZIP ---
zip_prices = DATA_DIR / "us-shareprices-daily.zip"
csv_prices = DATA_DIR / "us-shareprices-daily.csv"

if (not csv_prices.exists()) and zip_prices.exists():
    print("📦 Encontré us-shareprices-daily.zip y no existe el CSV. Descomprimiendo...")
    with zipfile.ZipFile(zip_prices, "r") as z:
        z.extractall(DATA_DIR)
    print("✓ Zip descomprimido en:", DATA_DIR)

# Verificación de existencia del CSV de precios
if not csv_prices.exists():
    print("❌ ERROR: No existe us-shareprices-daily.csv en /content/data")
    print("   - Si tienes el zip, asegúrate que se llame: us-shareprices-daily.zip")
    print("   - Archivos presentes (top 20):", [p.name for p in sorted(DATA_DIR.glob("*"))[:20]])
    raise FileNotFoundError("Falta us-shareprices-daily.csv (o zip no descomprimió correctamente).")

print(f"✓ prices CSV listo: {csv_prices.name} | size_MB={csv_prices.stat().st_size/(1024**2):.2f}\n")

# --- 2.3.2: Carga de los 6 CSV ---
files = {
    "industries": "industries.csv",
    "companies": "us-companies.csv",
    "balance_q": "us-balance-quarterly.csv",
    "income_q": "us-income-quarterly.csv",
    "cashflow_q": "us-cashflow-quarterly.csv",
    "prices_d": "us-shareprices-daily.csv",
}

dfs = {}
print("📚 Leyendo CSV (sep=';')...\n")

for name, fname in files.items():
    file_path = DATA_DIR / fname
    try:
        df = pd.read_csv(file_path, sep=";", low_memory=False)
        dfs[name] = df
        print(f"✓ {fname} cargado correctamente")
        print(f"  - {name}: {df.shape[0]:,} registros × {df.shape[1]} columnas\n")
    except FileNotFoundError:
        print(f"❌ ERROR: No se encuentra el archivo: {fname}")
        print(f"   Ruta esperada: {file_path}")
        raise

# --- 2.3.3: Sanity check mínimo del archivo grande (5 filas) ---
print("🔎 Sanity check rápido de prices (5 filas):")
df_prices_test = pd.read_csv(csv_prices, sep=";", nrows=5, low_memory=False)
display(df_prices_test)
print("✓ Columnas prices:", df_prices_test.columns.tolist())

# --- 2.3.4: Validación visual del cargue de los archivos ---
for name, df in dfs.items():
    print("\n" + "="*90)
    print(f"{name} | shape: {df.shape[0]:,} × {df.shape[1]}")
    display(df.head(5))


📦 Encontré us-shareprices-daily.zip y no existe el CSV. Descomprimiendo...
✓ Zip descomprimido en: /content/data
✓ prices CSV listo: us-shareprices-daily.csv | size_MB=413.49

📚 Leyendo CSV (sep=';')...

✓ industries.csv cargado correctamente
  - industries: 74 registros × 3 columnas

✓ us-companies.csv cargado correctamente
  - companies: 6,525 registros × 11 columnas

✓ us-balance-quarterly.csv cargado correctamente
  - balance_q: 52,098 registros × 30 columnas

✓ us-income-quarterly.csv cargado correctamente
  - income_q: 52,106 registros × 28 columnas

✓ us-cashflow-quarterly.csv cargado correctamente
  - cashflow_q: 52,103 registros × 28 columnas

✓ us-shareprices-daily.csv cargado correctamente
  - prices_d: 6,210,379 registros × 11 columnas

🔎 Sanity check rápido de prices (5 filas):


,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
0,A,45846,2020-03-30,71.06,73.18,71.06,72.67,69.86,1486203,0.18,309651359
1,A,45846,2020-03-31,72.34,72.80,70.50,71.62,68.85,1822122,NaN,309651359
2,A,45846,2020-04-01,69.47,70.23,68.15,68.92,66.26,2173595,NaN,309651359
3,A,45846,2020-04-02,68.27,72.45,68.14,72.29,69.50,1840311,NaN,309651359
4,A,45846,2020-04-03,71.71,72.33,69.66,70.42,67.70,2052642,NaN,309651359


✓ Columnas prices: ['Ticker', 'SimFinId', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj. Close', 'Volume', 'Dividend', 'Shares Outstanding']

industries | shape: 74 × 3


,IndustryId,Industry,Sector
0,100001,Industrial Products,Industrials
1,100002,Business Services,Industrials
2,100003,Engineering & Construction,Industrials
3,100004,Waste Management,Industrials
4,100005,Industrial Distribution,Industrials



companies | shape: 6,525 × 11


...

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
0,A,45846,USD,2020,Q2,2020-04-30,2020-06-01,2021-06-01,309000000.0,312000000.0,1.238000e+09,-581000000.0,657000000.0,-555000000.0,-358000000.0,-197000000.0,NaN,102000000.0,19000000.0,-17000000.0,121000000.0,NaN,121000000,-20000000.0,101000000,NaN,101000000,101000000
1,A,45846,USD,2020,Q3,2020-07-31,2020-09-01,2021-09-01,309000000.0,312000000.0,1.261000e+09,-592000000.0,669000000.0,-439000000.0,-347000000.0,-92000000.0,NaN,230000000.0,-11000000.0,-18000000.0,219000000.0,NaN,219000000,-20000000.0,199000000,NaN,199000000,199000000
2,A,45846,USD,2020,Q4,2020-10-31,2020-12-18,2021-09-01,308000000.0,311000000.0,1.483000e+09,-695000000.0,788000000.0,-489000000.0,-387000000.0,-102000000.0,NaN,299000000.0,-16000000.0,-18000000.0,283000000.0,NaN,283000000,-61000000.0,222000000,NaN,222000000,222000000
3,A,45846,USD,2021,Q1,2021-01-31,2021-03-02,2022-03-03,306000000.0,309000000.0,1.548000e+09,-710000000.0,838000000.0,-510000000.0,-407000000.0,-103000000.0,NaN,328000000.0,-16000000.0,-19000000.0,312000000.0,NaN,312000000,-24000000.0,288000000,NaN,288000000,288000000
4,A,45846,USD,2021,Q2,2021-04-30,2021-06-01,2022-05-31,306000000.0,306000000.0,1.525000e+09,-708000000.0,817000000.0,-529000000.0,-420000000.0,-109000000.0,NaN,288000000.0,-15000000.0,-19000000.0,273000000.0,NaN,273000000,-57000000.0,216000000,NaN,216000000,216000000



cashflow_q | shape: 52,103 × 28


,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Change in Working Capital,Change in Accounts Receivable,Change in Inventories,Change in Accounts Payable,Change in Other,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,45846,USD,2020,Q2,2020-04-30,2020-06-01,2021-03-02,309000000.0,312000000.0,101000000.0,76000000.0,98000000.0,38000000.0,65000000.0,-53000000.0,5000000.0,21000000.0,313000000.0,-33000000.0,NaN,NaN,-53000000.0,-55000000.0,25000000.0,-126000000.0,-156000000.0,97000000
1,A,45846,USD,2020,Q3,2020-07-31,2020-09-01,2021-06-01,309000000.0,312000000.0,199000000.0,77000000.0,34000000.0,-20000000.0,-24000000.0,-1000000.0,-25000000.0,30000000.0,290000000.0,-24000000.0,NaN,NaN,-32000000.0,-56000000.0,-161000000.0,-9000000.0,-231000000.0,35000000
2,A,45846,USD,2020,Q4,2020-10-31,2020-12-18,2021-09-01,308000000.0,311000000.0,222000000.0,76000000.0,61000000.0,18000000.0,NaN,NaN,NaN,NaN,377000000.0,-27000000.0,NaN,NaN,-27000000.0,-55000000.0,35000000.0,-246000000.0,-269000000.0,83000000
3,A,45846,USD,2021,Q1,2021-01-31,2021-03-02,2022-03-03,306000000.0,309000000.0,288000000.0,76000000.0,80000000.0,-206000000.0,-31000000.0,-35000000.0,43000000.0,-183000000.0,238000000.0,-41000000.0,NaN,NaN,-42000000.0,-59000000.0,134000000.0,-319000000.0,-316000000.0,-111000000
4,A,45846,USD,2021,Q2,2021-04-30,2021-06-01,2022-03-03,306000000.0,306000000.0,216000000.0,77000000.0,37000000.0,142000000.0,14000000.0,-45000000.0,8000000.0,165000000.0,472000000.0,-31000000.0,NaN,-547000000.0,-587000000.0,-59000000.0,427000000.0,-194000000.0,166000000.0,51000000



prices_d | shape: 6,210,379 × 11


,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
0,A,45846,2020-03-30,71.06,73.18,71.06,72.67,69.86,1486203,0.18,309651359.0
1,A,45846,2020-03-31,72.34,72.80,70.50,71.62,68.85,1822122,NaN,309651359.0
2,A,45846,2020-04-01,69.47,70.23,68.15,68.92,66.26,2173595,NaN,309651359.0
3,A,45846,2020-04-02,68.27,72.45,68.14,72.29,69.50,1840311,NaN,309651359.0
4,A,45846,2020-04-03,71.71,72.33,69.66,70.42,67.70,2052642,NaN,309651359.0


## 2.4 Exploración Estadística Básica

Inspeccionar tipos de datos, valores faltantes y estadísticos descriptivos básicos para cada dataset, como verificación inicial antes de limpieza.


In [5]:
# 2.4 Exploración Estadística Básica (completa y robusta para prices_d)

SEP_LINE = "\n" + "="*80 + "\n"
SAMPLE_N = 100_000   # muestra para datasets grandes (ej. prices_d)
TOP_NULLS = 20       # top columnas con más nulos
TOP_EXAMPLES = 5     # top valores por categoría

for name, df in dfs.items():
    # Vista a analizar: para prices_d muy grande, usamos muestra (evita uso alto de RAM/tiempo)
    if name == "prices_d" and len(df) > SAMPLE_N:
        df_view = df.sample(SAMPLE_N, random_state=42)
        view_note = f"(vista: muestra aleatoria n={SAMPLE_N:,})"
    else:
        df_view = df
        view_note = "(vista: completo)"

    print("\n" + "#"*90)
    print(f"DATASET: {name} {view_note}")
    print(f"Shape original: {df.shape[0]:,} × {df.shape[1]}  |  Shape vista: {df_view.shape[0]:,} × {df_view.shape[1]}")
    print("#"*90)

    # 1) Información general sobre el DataFrame
    print("Información del Dataset:")
    buf = io.StringIO()
    df_view.info(buf=buf)
    print(buf.getvalue())
    print(SEP_LINE)

    # 2) Resumen estadístico de columnas numéricas
    print("Resumen Estadístico (numéricas):")
    df_num = df_view.select_dtypes(include=[np.number])
    if df_num.shape[1] == 0:
        print("ℹ️ No hay columnas numéricas para describe().")
    else:
        display(df_num.describe().T)
    print(SEP_LINE)

    # 3) Verifica valores nulos (conteo + % + top-N)
    print("Valores Nulos por Columna (top):")
    null_count = df_view.isnull().sum()
    null_count = null_count[null_count > 0].sort_values(ascending=False)

    if len(null_count) == 0:
        print("✓ Sin nulos")
    else:
        null_pct = (null_count / len(df_view) * 100).round(2)
        null_summary = pd.DataFrame({"null_count": null_count, "null_pct": null_pct}).head(TOP_NULLS)
        display(null_summary)
    print(SEP_LINE)

    # 4) Para columnas categóricas: nunique + ejemplos top-N
    print("Valores Únicos en Columnas Categóricas:")
    obj_cols = df_view.select_dtypes(include=["object"]).columns.tolist()

    if len(obj_cols) == 0:
        print("ℹ️ No hay columnas categóricas (object).")
    else:
        for col in obj_cols:
            nunq = df_view[col].nunique(dropna=True)
            print(f"\n{col}: {nunq:,} valores únicos")

            examples = df_view[col].value_counts(dropna=True).head(TOP_EXAMPLES).to_dict()
            print(f"Ejemplos (top {TOP_EXAMPLES}): {examples}")


##########################################################################################
DATASET: industries (vista: completo)
Shape original: 74 × 3  |  Shape vista: 74 × 3
##########################################################################################
Información del Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   IndustryId  74 non-null     int64 
 1   Industry    74 non-null     object
 2   Sector      74 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.9+ KB



Resumen Estadístico (numéricas):


,count,mean,std,min,25%,50%,75%,max
IndustryId,74.0,104329.797297,3293.071327,100001.0,102001.25,104002.5,107002.75,111001.0




Valores Nulos por Columna (top):
✓ Sin nulos


Valores Únicos en Columnas Categóricas:

Industry: 74 valores únicos
Ejemplos (top 5): {'Industrial Products': 1, 'Business Services': 1, 'Engineering & Construction': 1, 'Waste Management': 1, 'Industrial Distribution': 1}

Sector: 12 valores únicos
Ejemplos (top 5): {'Industrials': 13, 'Consumer Cyclical': 11, 'Healthcare': 8, 'Financial Services': 8, 'Energy': 7}

##########################################################################################
DATASET: companies (vista: completo)
Shape original: 6,525 × 11  |  Shape vista: 6,525 × 11
##########################################################################################
Información del Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6525 entries, 0 to 6524
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ticker                         6488 non-null   obj

,count,mean,std,min,25%,50%,75%,max
SimFinId,6525.0,6.991677e+06,6.783715e+06,18.0,663113.0,6744552.0,12444309.0,19976457.0
IndustryId,6224.0,1.040509e+05,2.817199e+03,100001.0,101004.0,104002.0,106002.0,111001.0
End of financial year (month),6492.0,1.098460e+01,2.656196e+00,1.0,12.0,12.0,12.0,12.0
Number Employees,5700.0,7.500325e+03,3.134010e+04,0.0,136.0,880.5,3700.0,1298000.0
CIK,6513.0,1.270512e+06,5.284349e+05,0.0,928054.0,1402436.0,1697862.0,2079173.0




Valores Nulos por Columna (top):


...



Valores Únicos en Columnas Categóricas:

Ticker: 3,704 valores únicos
Ejemplos (top 5): {'APOG': 21, 'COST': 21, 'ENTA': 20, 'ENSG': 20, 'ENS': 20}

Currency: 1 valores únicos
Ejemplos (top 5): {'USD': 52103}

Fiscal Period: 4 valores únicos
Ejemplos (top 5): {'Q1': 13134, 'Q2': 13061, 'Q3': 13033, 'Q4': 12875}

Report Date: 66 valores únicos
Ejemplos (top 5): {'2022-03-31': 2547, '2022-06-30': 2543, '2023-03-31': 2532, '2022-09-30': 2524, '2023-06-30': 2502}

Publish Date: 1,389 valores únicos
Ejemplos (top 5): {'2024-08-08': 390, '2024-11-07': 375, '2021-08-05': 371, '2020-11-05': 371, '2022-08-04': 370}

Restated Date: 1,302 valores únicos
Ejemplos (top 5): {'2024-05-09': 703, '2025-05-08': 653, '2023-05-04': 617, '2023-05-09': 611, '2022-05-05': 536}

##########################################################################################
DATASET: prices_d (vista: muestra aleatoria n=100,000)
Shape original: 6,210,379 × 11  |  Shape vista: 100,000 × 11
#########################

,count,mean,std,min,25%,50%,75%,max
SimFinId,100000.0,6.875920e+06,6.628685e+06,18.0,682408.00,6.767429e+06,1.181977e+07,1.993759e+07
Open,100000.0,3.594203e+04,1.710728e+06,0.0,7.40,1.947000e+01,5.207000e+01,1.000000e+08
High,100000.0,3.648431e+04,1.723215e+06,0.0,7.60,1.988000e+01,5.302250e+01,1.000000e+08
Low,100000.0,3.555576e+04,1.704572e+06,0.0,7.19,1.907500e+01,5.120000e+01,1.000000e+08
Close,100000.0,3.598227e+04,1.713159e+06,0.0,7.39,1.948000e+01,5.209000e+01,1.000000e+08
Adj. Close,100000.0,3.597991e+04,1.713159e+06,0.0,6.92,1.810000e+01,4.892000e+01,1.000000e+08
Volume,100000.0,1.621350e+06,1.625400e+07,0.0,32946.25,2.391405e+05,9.434645e+05,3.352070e+09
Dividend,596.0,5.137081e-01,1.447287e+00,0.0,0.12,2.600000e-01,4.900000e-01,2.819000e+01
Shares Outstanding,91469.0,4.981733e+11,5.679120e+13,0.0,17194000.00,5.009359e+07,1.417000e+08,6.667887e+15




Valores Nulos por Columna (top):


,null_count,null_pct
Dividend,99404,99.40
Shares Outstanding,8531,8.53
Ticker,13,0.01




Valores Únicos en Columnas Categóricas:

Ticker: 5,792 valores únicos
Ejemplos (top 5): {'FTNT': 38, 'WRLD': 37, 'RVPH': 34, 'FBIO': 34, 'NMTC': 34}

Date: 1,237 valores únicos
Ejemplos (top 5): {'2023-08-17': 111, '2023-09-20': 111, '2022-06-03': 111, '2022-01-05': 110, '2023-01-25': 110}


# 3. Preprocesamiento y limpieza de datos

3.1 Manejo de valores nulos
Aplicar reglas de manejo de nulos para asegurar llaves completas, definir el universo (empresas con ticker), marcar IndustryId conocido/desconocido y preparar columnas canónicas mínimas para el MVP sin imputar valores financieros.

In [6]:
# 3.1 Manejo de valores nulos (MVP, sin inventar datos)

dfs_proc = {name: df.copy() for name, df in dfs.items()}

print("🧹 3.1 Manejo de valores nulos \n")

# -------------------------
# 1) COMPANIES: universo + Industry flag + Company Name display
# -------------------------
companies = dfs_proc["companies"].copy()

# Normalizar strings
companies["Ticker"] = companies["Ticker"].astype(str).str.strip()
companies.loc[companies["Ticker"].isin(["", "nan", "None"]), "Ticker"] = np.nan

# Regla: excluir compañías sin ticker
before = len(companies)
companies = companies.dropna(subset=["Ticker"])
removed = before - len(companies)
print(f"companies: removidas sin Ticker -> {removed:,}")

# Industry flag (no elimina)
if "IndustryId" in companies.columns:
    companies["has_industry"] = companies["IndustryId"].notna()
else:
    companies["has_industry"] = False
    print("⚠️ companies: no encontré IndustryId, se marca has_industry=False")

# Company Name: imputación SOLO para display
if "Company Name" in companies.columns:
    companies["Company Name"] = companies["Company Name"].fillna("Unknown")

dfs_proc["companies"] = companies

# -------------------------
# 2) INDUSTRIES: sin cambios (0 nulos en tu data)
# -------------------------
# (Opcional) validar que IndustryId no sea nulo
industries = dfs_proc["industries"].copy()
industries = industries.dropna(subset=["IndustryId"])
dfs_proc["industries"] = industries

# -------------------------
# 3) FACTS Q: drop filas con llaves nulas (para joins)
# -------------------------
fact_q_keys = ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date"]

for t in ["balance_q", "income_q", "cashflow_q"]:
    dfq = dfs_proc[t].copy()

    missing = [c for c in fact_q_keys if c not in dfq.columns]
    if missing:
        raise KeyError(f"{t}: faltan columnas llave {missing}. Columnas: {dfq.columns.tolist()}")

    before = len(dfq)
    dfq = dfq.dropna(subset=fact_q_keys)
    print(f"{t}: removidas filas con NULL en llaves -> {before - len(dfq):,}")

    dfs_proc[t] = dfq

# -------------------------
# 4) PRICES D: drop filas sin llaves (SimFinId, Date)
# -------------------------
prices = dfs_proc["prices_d"].copy()
req_prices_keys = ["SimFinId", "Date"]
missing = [c for c in req_prices_keys if c not in prices.columns]
if missing:
    raise KeyError(f"prices_d: faltan columnas llave {missing}. Columnas: {prices.columns.tolist()}")

before = len(prices)
prices = prices.dropna(subset=req_prices_keys)
print(f"prices_d: removidas filas con NULL en llaves -> {before - len(prices):,}")

# Nota: NO exigimos Ticker en prices_d (join por SimFinId)
dfs_proc["prices_d"] = prices

# -------------------------
# 5) Preparar columnas canónicas mínimas (sin calcular aún ratios TTMs)
#    Solo creamos alias canónicos para MVP (útil para trazabilidad)
# -------------------------

# BALANCE canónicas mínimas para MVP
bal = dfs_proc["balance_q"].copy()
bal["TCA"] = bal["Total Current Assets"]
bal["TCL"] = bal["Total Current Liabilities"]
bal["CASH"] = bal["Cash, Cash Equivalents & Short Term Investments"]
bal["STD"] = bal["Short Term Debt"]
bal["LTD"] = bal["Long Term Debt"]
bal["TE"]  = bal["Total Equity"]

# TotalDebt: solo si STD y LTD presentes (regla conservadora)
bal["TotalDebt"] = np.where(bal["STD"].notna() & bal["LTD"].notna(), bal["STD"] + bal["LTD"], np.nan)

# NetDebt: solo si TotalDebt y CASH presentes
bal["NetDebt"] = np.where(bal["TotalDebt"].notna() & bal["CASH"].notna(), bal["TotalDebt"] - bal["CASH"], np.nan)

dfs_proc["balance_q"] = bal

# INCOME canónicas mínimas para MVP
inc = dfs_proc["income_q"].copy()
inc["Revenue_Q"] = inc["Revenue"]
inc["EBIT_Q"] = inc["Operating Income (Loss)"]
inc["NetIncomeCommon_Q"] = inc["Net Income (Common)"]
dfs_proc["income_q"] = inc

# CASHFLOW canónicas mínimas para MVP
cf = dfs_proc["cashflow_q"].copy()
cf["CFO_Q"] = cf["Net Cash from Operating Activities"]
cf["ChangeFixedAssets_Q"] = cf["Change in Fixed Assets & Intangibles"]
cf["CapexProxy_Q"] = np.where(cf["ChangeFixedAssets_Q"].notna(), -cf["ChangeFixedAssets_Q"], np.nan)
cf["FCF_Q"] = np.where(cf["CFO_Q"].notna() & cf["CapexProxy_Q"].notna(), cf["CFO_Q"] - cf["CapexProxy_Q"], np.nan)
dfs_proc["cashflow_q"] = cf

# PRICES canónicas mínimas para MVP
pr = dfs_proc["prices_d"].copy()
pr["Price_D"] = pr["Adj. Close"]
pr["SharesOut_D"] = pr["Shares Outstanding"]  # ffill lo haremos al cargar a SQL
dfs_proc["prices_d"] = pr

# -------------------------
# 6) Verificación de nulos después de "limpieza estructural"
# -------------------------
print("\n📌 Valores nulos después de limpieza (top 10 por dataset):")
TOP = 10
for name, df in dfs_proc.items():
    print("\n" + "-"*90)
    print(f"{name} | shape: {df.shape[0]:,} × {df.shape[1]}")
    nulls = df.isnull().sum()
    nulls = nulls[nulls > 0].sort_values(ascending=False).head(TOP)
    if len(nulls) == 0:
        print("✓ Sin nulos")
    else:
        display(pd.DataFrame({"null_count": nulls, "null_pct": (nulls/len(df)*100).round(2)}))

🧹 3.1 Manejo de valores nulos (MVP)

companies: removidas sin Ticker -> 37
balance_q: removidas filas con NULL en llaves -> 0
income_q: removidas filas con NULL en llaves -> 0
cashflow_q: removidas filas con NULL en llaves -> 0
prices_d: removidas filas con NULL en llaves -> 0

📌 Valores nulos después de limpieza (top 10 por dataset):

------------------------------------------------------------------------------------------
industries | shape: 74 × 3
✓ Sin nulos

------------------------------------------------------------------------------------------
companies | shape: 6,488 × 12


,null_count,null_pct
ISIN,1145,17.65
Number Employees,789,12.16
IndustryId,265,4.08
Business Summary,258,3.98
CIK,11,0.17



------------------------------------------------------------------------------------------
balance_q | shape: 52,098 × 38


,null_count,null_pct
Long Term Investments & Receivables,40123,77.01
Treasury Stock,34481,66.18
NetDebt,26875,51.59
TotalDebt,26811,51.46
STD,24366,46.77
Short Term Debt,24366,46.77
Inventories,23736,45.56
LTD,14467,27.77
Long Term Debt,14467,27.77
Accounts & Notes Receivable,10731,20.60



------------------------------------------------------------------------------------------
income_q | shape: 52,106 × 31


,null_count,null_pct
Net Extraordinary Gains (Losses),48813,93.68
Depreciation & Amortization,31091,59.67
Research & Development,26714,51.27
Abnormal Gains (Losses),23809,45.69
Cost of Revenue,11339,21.76
Gross Profit,11326,21.74
"Income Tax (Expense) Benefit, Net",11200,21.49
"Interest Expense, Net",6771,12.99
Revenue_Q,5647,10.84
Revenue,5647,10.84



------------------------------------------------------------------------------------------
cashflow_q | shape: 52,103 × 32


,null_count,null_pct
Change in Inventories,51610,99.05
Change in Accounts Payable,51493,98.83
Change in Accounts Receivable,51382,98.62
Change in Other,51158,98.19
Net Cash from Acquisitions & Divestitures,35258,67.67
Dividends Paid,33962,65.18
Net Change in Long Term Investment,33561,64.41
Cash from (Repayment of) Debt,13313,25.55
Cash from (Repurchase of) Equity,12131,23.28
FCF_Q,4422,8.49



------------------------------------------------------------------------------------------
prices_d | shape: 6,210,379 × 13


,null_count,null_pct
Dividend,6174011,99.41
Shares Outstanding,528541,8.51
SharesOut_D,528541,8.51
Ticker,662,0.01


# 3.2 Corrección de tipos de datos

MD (una línea para 3.2): Estandarizar tipos de datos (fechas, periodos y numéricos) para asegurar joins temporales correctos, cálculos robustos y carga consistente en SQLite.



In [7]:
# 3.2 Conversión de tipos

dfs_proc2 = {name: df.copy() for name, df in dfs_proc.items()}

def to_date(df, col):
    df[col] = pd.to_datetime(df[col], errors="coerce")

def to_num(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

# companies
comp = dfs_proc2["companies"].copy()
if "IndustryId" in comp.columns:
    comp["IndustryId"] = pd.to_numeric(comp["IndustryId"], errors="coerce").astype("Int64")
dfs_proc2["companies"] = comp

# facts trimestrales
for t in ["balance_q", "income_q", "cashflow_q"]:
    dfq = dfs_proc2[t].copy()

    for c in ["Report Date", "Publish Date", "Restated Date"]:
        if c in dfq.columns:
            to_date(dfq, c)

    if "Fiscal Year" in dfq.columns:
        dfq["Fiscal Year"] = pd.to_numeric(dfq["Fiscal Year"], errors="coerce").astype("Int64")
    if "Fiscal Period" in dfq.columns:
        dfq["Fiscal Period"] = dfq["Fiscal Period"].astype("category")

    dfs_proc2[t] = dfq

# prices
pr = dfs_proc2["prices_d"].copy()
if "Date" in pr.columns:
    to_date(pr, "Date")
dfs_proc2["prices_d"] = pr

# numéricos clave (indicadores seleccionados)
bal = dfs_proc2["balance_q"].copy()
to_num(bal, [
    "Total Current Assets", "Total Current Liabilities",
    "Cash, Cash Equivalents & Short Term Investments",
    "Short Term Debt", "Long Term Debt", "Total Equity",
    "Total Assets", "Total Liabilities & Equity"
])
dfs_proc2["balance_q"] = bal

inc = dfs_proc2["income_q"].copy()
to_num(inc, ["Revenue", "Operating Income (Loss)", "Net Income (Common)"])
dfs_proc2["income_q"] = inc

cf = dfs_proc2["cashflow_q"].copy()
to_num(cf, ["Net Cash from Operating Activities", "Change in Fixed Assets & Intangibles"])
dfs_proc2["cashflow_q"] = cf

pr = dfs_proc2["prices_d"].copy()
to_num(pr, ["Adj. Close", "Shares Outstanding"])
dfs_proc2["prices_d"] = pr

print("Tipos de datos después de conversión:\n")

check_cols = {
    "companies": ["SimFinId", "Ticker", "IndustryId", "has_industry", "Company Name"],
    "balance_q": ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date",
                  "Total Current Assets", "Total Current Liabilities",
                  "Cash, Cash Equivalents & Short Term Investments",
                  "Short Term Debt", "Long Term Debt", "Total Equity"],
    "income_q": ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date",
                 "Revenue", "Operating Income (Loss)", "Net Income (Common)"],
    "cashflow_q": ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date",
                   "Net Cash from Operating Activities", "Change in Fixed Assets & Intangibles"],
    "prices_d": ["SimFinId", "Date", "Adj. Close", "Shares Outstanding"],
}

for name, cols in check_cols.items():
    df = dfs_proc2[name]
    cols_present = [c for c in cols if c in df.columns]
    print("\n" + "-"*90)
    print(name, "| columnas revisadas:", cols_present)
    print(df[cols_present].dtypes)

dfs_proc = dfs_proc2

Tipos de datos después de conversión:


------------------------------------------------------------------------------------------
companies | columnas revisadas: ['SimFinId', 'Ticker', 'IndustryId', 'has_industry', 'Company Name']
SimFinId         int64
Ticker          object
IndustryId       Int64
has_industry      bool
Company Name    object
dtype: object

------------------------------------------------------------------------------------------
balance_q | columnas revisadas: ['SimFinId', 'Fiscal Year', 'Fiscal Period', 'Report Date', 'Total Current Assets', 'Total Current Liabilities', 'Cash, Cash Equivalents & Short Term Investments', 'Short Term Debt', 'Long Term Debt', 'Total Equity']
SimFinId                                                    int64
Fiscal Year                                                 Int64
Fiscal Period                                            category
Report Date                                        datetime64[ns]
Total Current Assets              

# 3.3 Normalización y estandarización

Normalizar campos categóricos y de texto (mayúsculas/minúsculas, espacios, valores vacíos) para evitar duplicidades lógicas y mejorar consistencia en joins, filtros y consultas del agente.

In [8]:
# 3.3 Normalizaciones (categóricas/texto)

dfs_proc3 = {name: df.copy() for name, df in dfs_proc.items()}

def normalize_text_series(s: pd.Series) -> pd.Series:
    # Normaliza: trim, colapsa espacios, estandariza vacío -> NA
    s2 = s.astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
    s2 = s2.replace({"": pd.NA, "nan": pd.NA, "None": pd.NA, "NaN": pd.NA})
    return s2

# -------------------------
# companies: Ticker, Market, Main Currency, Company Name
# -------------------------
comp = dfs_proc3["companies"].copy()

if "Ticker" in comp.columns:
    comp["Ticker"] = normalize_text_series(comp["Ticker"]).str.upper()

if "Market" in comp.columns:
    comp["Market"] = normalize_text_series(comp["Market"]).str.lower()

if "Main Currency" in comp.columns:
    comp["Main Currency"] = normalize_text_series(comp["Main Currency"]).str.upper()

if "Company Name" in comp.columns:
    comp["Company Name"] = normalize_text_series(comp["Company Name"])

dfs_proc3["companies"] = comp

# -------------------------
# facts trimestrales: Ticker, Currency, Fiscal Period
# -------------------------
for t in ["balance_q", "income_q", "cashflow_q"]:
    dfq = dfs_proc3[t].copy()

    if "Ticker" in dfq.columns:
        dfq["Ticker"] = normalize_text_series(dfq["Ticker"]).str.upper()

    if "Currency" in dfq.columns:
        dfq["Currency"] = normalize_text_series(dfq["Currency"]).str.upper()

    if "Fiscal Period" in dfq.columns:
        dfq["Fiscal Period"] = normalize_text_series(dfq["Fiscal Period"]).str.upper()
        # opcional: asegurar valores esperados Q1-Q4 (sin borrar, solo normaliza)

    dfs_proc3[t] = dfq

# -------------------------
# prices: Ticker
# -------------------------
pr = dfs_proc3["prices_d"].copy()
if "Ticker" in pr.columns:
    pr["Ticker"] = normalize_text_series(pr["Ticker"]).str.upper()
dfs_proc3["prices_d"] = pr

# -------------------------
# Verificación: cardinalidad + top values por dataset
# -------------------------
print("Valores únicos después de normalización (top 10 valores por columna):\n")

for name, df in dfs_proc3.items():
    print("\n" + "="*110)
    print(f"DATASET: {name} | shape: {df.shape[0]:,} × {df.shape[1]}")

    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    if len(cat_cols) == 0:
        print("ℹ️ No hay columnas object/category.")
        continue

    for col in cat_cols:
        nunq = df[col].nunique(dropna=True)
        print(f"\n{col}: {nunq:,} valores únicos")
        display(df[col].value_counts(dropna=True).head(10))

dfs_proc = dfs_proc3

Valores únicos después de normalización (top 10 valores por columna):


DATASET: industries | shape: 74 × 3

Industry: 74 valores únicos


,count
Industry,
Industrial Products,1
Business Services,1
Engineering & Construction,1
Waste Management,1
Industrial Distribution,1
Airlines,1
Consulting & Outsourcing,1
Aerospace & Defense,1
Farm & Construction Machinery,1



Sector: 12 valores únicos


,count
Sector,
Industrials,13
Consumer Cyclical,11
Healthcare,8
Financial Services,8
Energy,7
Basic Materials,7
Consumer Defensive,6
Technology,5
Business Services,4



DATASET: companies | shape: 6,488 × 12

Ticker: 6,488 valores únicos


...

,count
Currency,
USD,52103



Fiscal Period: 4 valores únicos


,count
Fiscal Period,
Q1,13134
Q2,13061
Q3,13033
Q4,12875



DATASET: prices_d | shape: 6,210,379 × 13

Ticker: 5,868 valores únicos


,count
Ticker,
ZYME,1237
A,1237
AA,1237
ZWS,1237
ZVRA,1237
ZIVO,1237
ZION,1237
ZEUS,1237
ABR,1237


# 3.4 Eliminación de duplicados

Detectar y gestionar duplicados según el grano de cada tabla (dimensiones y hechos), preservando la versión más reciente cuando existan reexpresiones o múltiples publicaciones.

In [9]:
# 3.4 Eliminación de duplicados (por grano)

dfs_proc4 = {name: df.copy() for name, df in dfs_proc.items()}

print("🧽 3.4 Eliminación de duplicados\n")

# Definición de llaves por tabla (grano)
keys = {
    "industries": ["IndustryId"],
    "companies": ["SimFinId"],  # se asume 1 fila por SimFinId
    "balance_q": ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date"],
    "income_q": ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date"],
    "cashflow_q": ["SimFinId", "Fiscal Year", "Fiscal Period", "Report Date"],
    "prices_d": ["SimFinId", "Date"],
}

# Orden de preferencia para conservar filas (si existe)
# En trimestrales, preferimos la fila con Restated Date más reciente; si no, Publish Date más reciente
order_cols_q = ["Restated Date", "Publish Date"]

# En prices no hay restatement; si hay duplicados, conservamos la última fila por orden natural (estable)
# (si existiera algún timestamp adicional se usaría aquí)

def dedup_table(name: str):
    df = dfs_proc4[name]
    k = keys[name]
    missing = [c for c in k if c not in df.columns]
    if missing:
        raise KeyError(f"{name}: faltan columnas llave {missing}")

    dup_count = df.duplicated(subset=k, keep=False).sum()
    print(f"{name}: duplicados por llave {k} -> {dup_count:,}")

    if dup_count == 0:
        return df

    # Hechos trimestrales: ordenar por Restated/Publish asc y quedarnos con la última (más reciente)
    if name in ["balance_q", "income_q", "cashflow_q"]:
        # Asegurar que columnas de orden existan; si no existen, se omiten
        sort_cols = [c for c in order_cols_q if c in df.columns]
        if len(sort_cols) > 0:
            df_sorted = df.sort_values(sort_cols)
            df_dedup = df_sorted.drop_duplicates(subset=k, keep="last")
        else:
            df_dedup = df.drop_duplicates(subset=k, keep="last")

    # Dimensiones y prices: mantener una fila por llave (última)
    else:
        df_dedup = df.drop_duplicates(subset=k, keep="last")

    print(f"{name}: filas después de dedup -> {len(df_dedup):,} (removidas {len(df)-len(df_dedup):,})")
    return df_dedup

for name in ["industries", "companies", "balance_q", "income_q", "cashflow_q", "prices_d"]:
    dfs_proc4[name] = dedup_table(name)

dfs_proc = dfs_proc4

🧽 3.4 Eliminación de duplicados

industries: duplicados por llave ['IndustryId'] -> 0
companies: duplicados por llave ['SimFinId'] -> 0
balance_q: duplicados por llave ['SimFinId', 'Fiscal Year', 'Fiscal Period', 'Report Date'] -> 0
income_q: duplicados por llave ['SimFinId', 'Fiscal Year', 'Fiscal Period', 'Report Date'] -> 0
cashflow_q: duplicados por llave ['SimFinId', 'Fiscal Year', 'Fiscal Period', 'Report Date'] -> 0
prices_d: duplicados por llave ['SimFinId', 'Date'] -> 0


## 4. Feature engineering
# 4.1 Feature 1 | CurrentRatio_Q
Construir un dataset trimestral integrado y derivar CurrentRatio_Q para evaluar liquidez de corto plazo y fragilidad financiera.

Estadísticas de la nueva feature (CurrentRatio_Q):
count    5.191100e+04
mean     1.933547e+01
std      2.267698e+03
min      8.905806e-07
25%      1.200087e+00
50%      2.036414e+00
75%      4.075795e+00
max      5.022542e+05
Name: CurrentRatio_Q, dtype: float64


# 4.2 Feature 2 | DebtToEquity_Q
Derivar DebtToEquity_Q para capturar apalancamiento financiero, clave en análisis de downside y detección de fragilidad.

In [ ]:
# 4.2 Feature 2: DebtToEquity_Q

# Columnas requeridas desde balance
for c in ["Short Term Debt", "Long Term Debt", "Total Equity"]:
    if c not in dfs_proc["balance_q"].columns:
        raise KeyError(f"Falta columna requerida en balance_q: {c}")

# Traer columnas a df_procesado (ya integrado por keys_q)
df_procesado = df_procesado.merge(
    dfs_proc["balance_q"][keys_q + ["Short Term Debt", "Long Term Debt", "Total Equity"]],
    on=keys_q, how="left"
)

STD = pd.to_numeric(df_procesado["Short Term Debt"], errors="coerce")
LTD = pd.to_numeric(df_procesado["Long Term Debt"], errors="coerce")
TE  = pd.to_numeric(df_procesado["Total Equity"], errors="coerce")

# Regla conservadora acordada: TotalDebt solo si ambas deudas existen
df_procesado["TotalDebt_Q"] = np.where(STD.notna() & LTD.notna(), STD + LTD, np.nan)

df_procesado["DebtToEquity_Q"] = np.where(
    df_procesado["TotalDebt_Q"].notna() & TE.notna() & (TE > 0),
    df_procesado["TotalDebt_Q"] / TE,
    np.nan
)

print("Estadísticas de la nueva feature (DebtToEquity_Q):")
print(df_procesado["DebtToEquity_Q"].describe())

### Imports + configuracion base

El acceso a los modelos de lenguaje de OpenAI requiere autenticación mediante una API key. En entornos de producción, estas credenciales se gestionan mediante sistemas especializados de gestión de secretos (como AWS Secrets Manager, Google Cloud Secret Manager, o HashiCorp Vault), pero para desarrollo y educación utilizamos `getpass` que solicita la clave de forma interactiva sin almacenarla permanentemente en el código fuente del notebook. Esta práctica es fundamental en seguridad: las API keys expuestas accidentalmente en repositorios públicos son detectadas automáticamente por bots y pueden generar cargos fraudulentos en cuestión de minutos.

Una vez configurada la variable de entorno `OPENAI_API_KEY`, todas las llamadas posteriores a la API de OpenAI en esta sesión de Python usarán automáticamente esta credencial, sin necesidad de pasarla explícitamente en cada invocación. Este patrón mantiene el código limpio y centraliza la gestión de autenticación.

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

DATA_DIR = "/content/data"

print("📁 DATA_DIR =", DATA_DIR)

# 4.2 Feature derivada
Derivar DebtToEquity_Q para capturar apalancamiento financiero, clave en análisis de downside y detección de fragilidad.

## 3. Carga y Exploración Inicial del Dataset (EDA)

El primer paso crítico en cualquier pipeline de datos es entender profundamente la materia prima con la que trabajaremos. Esta fase de Análisis Exploratorio de Datos (EDA) no es mera curiosidad estadística, sino una práctica esencial de ingeniería que informa todas las decisiones posteriores de transformación y modelado. Al entender la distribución de precios, podemos decidir cómo segmentar el mercado en categorías. Al analizar la cardinalidad de modelos y regiones, podemos estimar el tamaño de nuestras futuras tablas de lookup. Al inspeccionar el rango temporal de los datos, podemos diseñar estrategias apropiadas de análisis de tendencias.

Este dataset contiene registros de ventas de vehículos BMW desde 2010 hasta 2024, abarcando múltiples modelos, regiones geográficas, configuraciones de vehículos y segmentos de precio. Cada registro representa una transacción de venta con sus características asociadas. La riqueza de este dataset nos permitirá construir un sistema de inteligencia de negocio que puede responder desde preguntas simples ("¿cuál es el precio promedio por región?") hasta análisis sofisticados ("¿cómo ha evolucionado la adopción de vehículos eléctricos en mercados premium europeos entre 2020-2024?").

**Nota importante:** Asegúrate de subir el archivo `BMW sales data (2010-2024) (1) (1).csv` a la sección de archivos de Colab (icono de carpeta en el panel izquierdo) antes de ejecutar la siguiente celda.

In [ ]:
import pandas as pd
import numpy as np

# Cargamos el dataset de ventas BMW
# Ajusta la ruta si el nombre del archivo es diferente al subirlo
file_path = "BMW sales data (2010-2024) (1) (1).csv"

try:
    df = pd.read_csv(file_path)
    print("✓ Dataset cargado correctamente")
    print(f"  Dimensiones: {df.shape[0]:,} registros × {df.shape[1]} columnas")
except FileNotFoundError:
    print("ERROR: No se encuentra el archivo.")
    print("Por favor, sube el archivo CSV al entorno de Colab usando el panel de archivos.")
    raise

# Inspección inicial de la estructura
print("\n--- Primeras filas del dataset ---")
display(df.head(10))

In [ ]:
# Análisis de tipos de datos y valores faltantes
print("=== INFORMACIÓN DEL DATASET ===")
df.info()

print("\n=== ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS ===")
display(df.describe())

In [ ]:
# Exploración de variables categóricas clave
print("=== ANÁLISIS EXPLORATORIO DE VARIABLES CATEGÓRICAS ===")

print("\n--- Distribución de Modelos BMW ---")
model_dist = df['Model'].value_counts()
print(model_dist)
print(f"\nModelos únicos: {df['Model'].nunique()}")

print("\n--- Distribución Geográfica de Ventas ---")
region_dist = df['Region'].value_counts()
print(region_dist)

print("\n--- Distribución de Tipos de Combustible ---")
fuel_dist = df['Fuel_Type'].value_counts()
print(fuel_dist)
print(f"\nPorcentaje de vehículos eléctricos: {(df['Fuel_Type'] == 'Electric').sum() / len(df) * 100:.2f}%")

print("\n--- Distribución de Transmisión ---")
transmission_dist = df['Transmission'].value_counts()
print(transmission_dist)

print("\n--- Distribución Temporal ---")
year_dist = df['Year'].value_counts().sort_index()
print(f"Rango temporal: {df['Year'].min()} - {df['Year'].max()}")
print(f"Año con más ventas: {year_dist.idxmax()} ({year_dist.max():,} registros)")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Identificación de Oportunidades de Feature Engineering

A partir de esta exploración inicial, podemos identificar varias oportunidades para crear features que amplificarán la capacidad del agente SQL para responder preguntas de negocio. Observa que tenemos el año del vehículo pero no la antigüedad relativa, tenemos precio absoluto pero no segmentación de mercado, tenemos kilometraje y precio por separado pero no eficiencia de valor. Estas son las transformaciones que implementaremos en la siguiente sección.

También notamos que los modelos BMW tienen jerarquías implícitas: las series numéricas (3, 5, 7) representan segmentos de sedán progresivamente más premium, los modelos X representan SUVs, los modelos M son versiones de alto rendimiento, y los modelos i son la línea de vehículos eléctricos e híbridos. Hacer estas jerarquías explícitas en nuestro esquema de base de datos facilitará queries como "analiza la evolución de ventas en el segmento SUV" o "compara el rendimiento de modelos de alto rendimiento versus modelos estándar".

## 4. Pipeline de Preprocesamiento y Feature Engineering

Esta es la fase donde aplicamos transformaciones inteligentes que convierten datos crudos en información estructurada para análisis. El objetivo no es solo limpiar datos (este dataset ya está razonablemente limpio), sino enriquecer la representación con features derivadas que hacen explícito conocimiento implícito del dominio. Cada feature que creamos es una decisión de diseño que balancea complejidad de implementación versus utilidad analítica.

Construiremos nuestro pipeline como una serie de funciones modulares y componibles, donde cada función tiene una responsabilidad clara y puede testearse independientemente. Este enfoque no solo es buena práctica de ingeniería de software, sino que facilita enormemente el debugging cuando el agente SQL genera queries incorrectas debido a problemas en los datos subyacentes.

### Fase 1: Cálculo de Antigüedad del Vehículo

El año de manufactura es un dato factual, pero lo que realmente importa para análisis de pricing, demanda y depreciación es la antigüedad relativa del vehículo. Un auto de 2015 en el año 2024 tiene 9 años de antigüedad, lo cual afecta dramáticamente su valor de mercado. Esta feature también facilita queries temporales como "analiza vehículos con menos de 3 años de antigüedad" que de otro modo requerirían aritmética compleja en SQL con la fecha actual.

In [ ]:
from datetime import datetime

def calculate_vehicle_age(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula la antigüedad del vehículo basado en el año actual.

    Args:
        df: DataFrame con columna 'Year'

    Returns:
        DataFrame con nueva columna 'Vehicle_Age'
    """
    df_processed = df.copy()
    current_year = datetime.now().year

    # Calculamos antigüedad
    df_processed['Vehicle_Age'] = current_year - df_processed['Year']

    # Aseguramos que no haya valores negativos (errores de datos futuros)
    df_processed['Vehicle_Age'] = df_processed['Vehicle_Age'].clip(lower=0)

    return df_processed

# Aplicamos la transformación
df_processed = calculate_vehicle_age(df)

print("✓ Feature 'Vehicle_Age' calculada")
print("\n--- Distribución de Antigüedad de Vehículos ---")
print(df_processed['Vehicle_Age'].describe())
print(f"\nRango: {df_processed['Vehicle_Age'].min()} - {df_processed['Vehicle_Age'].max()} años")

### Fase 2: Segmentación de Mercado por Precio

Los precios absolutos en USD son útiles para comparaciones numéricas, pero para análisis estratégico de mercado necesitamos categorías que representen segmentos de consumidores. Un vehículo de $30,000 USD compite en un segmento fundamentalmente diferente al de uno de $150,000 USD. Crearemos cuatro categorías de segmentación: Budget (económico), Mid-range (rango medio), Premium (premium) y Luxury (lujo). Estos umbrales se basan en conocimiento de la industria automotriz sobre cómo BMW posiciona sus modelos.

Esta categorización permite queries semánticas como "analiza la adopción de vehículos eléctricos en el segmento premium" que serían extremadamente verbosas si tuviéramos que especificar rangos de precio en cada query.

In [ ]:
def create_price_segments(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea categorías de segmentación de mercado basadas en precio.

    Args:
        df: DataFrame con columna 'Price_USD'

    Returns:
        DataFrame con nueva columna 'Price_Segment'
    """
    df_processed = df.copy()

    # Definimos umbrales basados en la estructura de mercado BMW
    def categorize_price(price):
        if price < 50000:
            return 'Budget'
        elif price < 80000:
            return 'Mid-range'
        elif price < 110000:
            return 'Premium'
        else:
            return 'Luxury'

    df_processed['Price_Segment'] = df_processed['Price_USD'].apply(categorize_price)

    return df_processed

# Aplicamos segmentación
df_processed = create_price_segments(df_processed)

print("✓ Feature 'Price_Segment' creada")
print("\n--- Distribución de Segmentos de Precio ---")
segment_dist = df_processed['Price_Segment'].value_counts()
print(segment_dist)
print("\n--- Precio Promedio por Segmento ---")
avg_by_segment = df_processed.groupby('Price_Segment')['Price_USD'].mean().sort_values()
print(avg_by_segment)

### Fase 3: Métricas de Eficiencia y Valor Relativo

El precio absoluto y el kilometraje son datos importantes individualmente, pero su ratio captura una dimensión crítica de valor: ¿cuánto cuesta este vehículo por cada kilómetro ya recorrido? Esta métrica es fundamental para análisis de valor relativo en el mercado de autos usados. Un vehículo de $100,000 con 10,000 km tiene un precio por kilómetro muy diferente al de uno de $50,000 con 150,000 km.

También calcularemos una estimación simplificada de depreciación anual. En el mercado automotriz, los vehículos típicamente deprecian más agresivamente en los primeros años. Crearemos una métrica que capture esta tendencia de forma aproximada, útil para análisis comparativo aunque no pretende ser un modelo preciso de valuación.

In [ ]:
def calculate_value_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula métricas derivadas de valor y eficiencia.

    Args:
        df: DataFrame con columnas 'Price_USD', 'Mileage_KM', 'Vehicle_Age'

    Returns:
        DataFrame con nuevas columnas de métricas
    """
    df_processed = df.copy()

    # Métrica 1: Precio por Kilómetro (evitando división por cero)
    df_processed['Price_per_KM'] = np.where(
        df_processed['Mileage_KM'] > 0,
        df_processed['Price_USD'] / df_processed['Mileage_KM'],
        df_processed['Price_USD']  # Si kilometraje es 0, usamos precio directamente
    )
    df_processed['Price_per_KM'] = df_processed['Price_per_KM'].round(2)

    # Métrica 2: Depreciación Anual Estimada
    # Asumimos un precio de lista promedio y calculamos depreciación
    # Fórmula simplificada: (Precio Estimado Nuevo - Precio Actual) / Años
    estimated_new_price = df_processed.groupby('Model')['Price_USD'].transform('max')
    df_processed['Annual_Depreciation'] = np.where(
        df_processed['Vehicle_Age'] > 0,
        (estimated_new_price - df_processed['Price_USD']) / df_processed['Vehicle_Age'],
        0
    )
    df_processed['Annual_Depreciation'] = df_processed['Annual_Depreciation'].clip(lower=0).round(2)

    # Métrica 3: Kilometraje Anual Promedio
    df_processed['Annual_Mileage'] = np.where(
        df_processed['Vehicle_Age'] > 0,
        df_processed['Mileage_KM'] / df_processed['Vehicle_Age'],
        df_processed['Mileage_KM']
    )
    df_processed['Annual_Mileage'] = df_processed['Annual_Mileage'].round(0).astype(int)

    return df_processed

# Aplicamos cálculo de métricas
df_processed = calculate_value_metrics(df_processed)

print("✓ Métricas de valor calculadas: Price_per_KM, Annual_Depreciation, Annual_Mileage")
print("\n--- Estadísticas de Nuevas Métricas ---")
display(df_processed[['Price_per_KM', 'Annual_Depreciation', 'Annual_Mileage']].describe())

### Fase 4: Jerarquía de Modelos y Líneas de Producto

Los modelos BMW tienen una estructura jerárquica implícita que es fundamental para análisis estratégico. Las series numéricas (3, 5, 7) representan sedanes de lujo progresivamente más premium, con la Serie 3 siendo el modelo de entrada y la Serie 7 el flagship. Los modelos X (X1, X3, X5, X7) son SUVs con numeración similar indicando tamaño y posicionamiento. Los modelos M son versiones de alto rendimiento de los modelos estándar. Los modelos i (i3, i8, iX) representan la línea de vehículos eléctricos e híbridos de BMW.

Hacer estas jerarquías explícitas mediante una columna `Product_Line` permitirá al agente responder preguntas como "compara el rendimiento de ventas entre sedanes y SUVs" o "analiza la adopción de la línea eléctrica i versus modelos tradicionales". Sin esta categorización, el agente tendría que hacer pattern matching complejo sobre nombres de modelos, lo cual es propenso a errores.

In [ ]:
def categorize_product_line(df: pd.DataFrame) -> pd.DataFrame:
    """
    Categoriza modelos en líneas de producto basadas en nomenclatura BMW.

    Args:
        df: DataFrame con columna 'Model'

    Returns:
        DataFrame con nueva columna 'Product_Line'
    """
    df_processed = df.copy()

    def classify_model(model):
        model = str(model)
        if model.startswith('X'):
            return 'SUV'
        elif model.startswith('M'):
            return 'Performance'
        elif model.startswith('i') or model.startswith('I'):
            return 'Electric/Hybrid'
        elif 'Series' in model:
            return 'Sedan'
        else:
            return 'Other'

    df_processed['Product_Line'] = df_processed['Model'].apply(classify_model)

    return df_processed

# Aplicamos categorización
df_processed = categorize_product_line(df_processed)

print("✓ Feature 'Product_Line' creada")
print("\n--- Distribución de Líneas de Producto ---")
product_line_dist = df_processed['Product_Line'].value_counts()
print(product_line_dist)

print("\n--- Precio Promedio por Línea de Producto ---")
avg_price_by_line = df_processed.groupby('Product_Line')['Price_USD'].mean().sort_values(ascending=False)
print(avg_price_by_line)

### Fase 5: Normalización de Nombres de Columnas para SQL

Las convenciones de nomenclatura de columnas en el dataset original usan PascalCase con guiones bajos (por ejemplo, `Fuel_Type`, `Engine_Size_L`). Aunque técnicamente SQL puede manejar estos nombres, la convención estándar en bases de datos es usar lowercase con guiones bajos (snake_case). Esta normalización previene problemas de case sensitivity entre diferentes sistemas de bases de datos y hace que las queries generadas por el agente sean más idiomáticas y fáciles de leer.

In [ ]:
def normalize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza nombres de columnas a snake_case lowercase (convención SQL).

    Args:
        df: DataFrame con nombres de columnas a normalizar

    Returns:
        DataFrame con nombres de columnas normalizados
    """
    df_processed = df.copy()

    # Convertimos a lowercase y reemplazamos espacios
    df_processed.columns = [col.lower().replace(' ', '_') for col in df_processed.columns]

    return df_processed

# Aplicamos normalización
df_final = normalize_column_names(df_processed)

print("✓ Nombres de columnas normalizados a snake_case")
print("\n--- Columnas Finales del Dataset ---")
print(df_final.columns.tolist())

print("\n--- Vista Final del Dataset Procesado ---")
display(df_final.head(10))

### Resumen del Pipeline de Preprocesamiento

En esta fase hemos transformado nuestro dataset crudo en una representación enriquecida que maximiza la utilidad para análisis de negocio. Creamos cinco nuevas features que amplifican la capacidad del agente SQL para razonar sobre los datos: antigüedad de vehículo (facilita análisis temporal), segmentación de precio (permite análisis de mercado), métricas de valor (habilitan comparaciones de eficiencia), línea de producto (estructura jerarquía de modelos) y normalización de nombres (mejora idiomaticidad de SQL).

Cada una de estas transformaciones fue una decisión de diseño informada por comprensión del dominio automotriz. No simplemente aplicamos transformaciones estándar, sino que pensamos profundamente sobre qué preguntas de negocio son importantes y cómo estructurar los datos para facilitarlas. Esta es la esencia del feature engineering efectivo: traducir conocimiento de dominio en representaciones de datos que amplifican el razonamiento de máquinas.

## 5. Diseño del Esquema Relacional: Normalización Estratégica

Ahora enfrentamos decisiones arquitectónicas fundamentales sobre cómo estructurar nuestros datos en una base de datos relacional. Para un dataset de este tipo, tenemos dos opciones principales: mantener todo en una tabla desnormalizada (flat) que es simple pero redundante, o normalizar en múltiples tablas relacionadas que es más complejo pero más eficiente y expresivo.

Tomaremos un enfoque híbrido pragmático: crearemos una tabla principal de transacciones (`sales_transactions`) que contendrá la mayoría de los datos, junto con tablas de lookup para entidades que tienen alta cardinalidad y metadata asociada. Este diseño balancea simplicidad (menos JOINs necesarios para queries comunes) con eficiencia (no repetir strings largos millones de veces).

Nuestro esquema consistirá en:

**Tabla `sales_transactions`:** La tabla central que contiene cada transacción de venta con todas las features calculadas. Esta será el punto de entrada natural para la mayoría de las queries del agente.

**Tabla `models`:** Catálogo de modelos BMW con metadata adicional como línea de producto y segmento típico. Esta separación permite agregar información futura (como año de lanzamiento del modelo, si está discontinuado, etc.) sin tocar las transacciones.

**Tabla `regions`:** Aunque solo tenemos el nombre de la región en el dataset original, crearemos esta tabla para facilitar futuras extensiones (como agrupar regiones en continentes o zonas económicas).

**Vista `sales_summary`:** Una vista materializada que pre-calcula estadísticas agregadas frecuentemente consultadas. Las vistas son como "queries guardadas" que simplifican dramáticamente la generación de SQL por parte del agente.

Una nota importante sobre este diseño: en un sistema de producción real con millones de registros y múltiples fuentes de datos, normalizaríamos más agresivamente y tendríamos esquemas de dimensiones (star schema o snowflake schema). Para este ejercicio educativo, priorizamos claridad y facilidad de uso para el agente sobre optimización extrema.

In [ ]:
from sqlalchemy import create_engine, text

# Creamos el motor de base de datos SQLite
db_path = "bmw_sales_intelligence.db"
engine = create_engine(f"sqlite:///{db_path}")

print(f"✓ Motor de base de datos creado: {db_path}")

# Tabla principal: sales_transactions
# Esta contiene todas las transacciones con sus features calculadas
df_final.to_sql('sales_transactions', engine, if_exists='replace', index=False)
print("  ✓ Tabla 'sales_transactions' creada")
print(f"    Registros: {len(df_final):,}")

### Creación de Tablas de Lookup y Catálogos

Ahora crearemos tablas auxiliares que proporcionan metadata estructurada sobre entidades clave. Estas tablas son relativamente pequeñas pero mejoran significativamente la expresividad de queries al permitir JOINs informativos y proporcionar puntos de extensión para metadata futura.

In [ ]:
# Tabla de catálogo de modelos
models_catalog = df_final[['model', 'product_line']].drop_duplicates()
models_catalog = models_catalog.reset_index(drop=True)
models_catalog.insert(0, 'model_id', range(1, len(models_catalog) + 1))

models_catalog.to_sql('models', engine, if_exists='replace', index=False)
print("  ✓ Tabla 'models' creada")
print(f"    Modelos únicos: {len(models_catalog)}")

# Tabla de regiones
regions_catalog = pd.DataFrame({
    'region_id': range(1, df_final['region'].nunique() + 1),
    'region_name': sorted(df_final['region'].unique())
})

regions_catalog.to_sql('regions', engine, if_exists='replace', index=False)
print("  ✓ Tabla 'regions' creada")
print(f"    Regiones únicas: {len(regions_catalog)}")

print("\n--- Catálogo de Modelos ---")
display(models_catalog)

print("\n--- Catálogo de Regiones ---")
display(regions_catalog)

### Creación de Vista Agregada para Análisis Rápido

Las vistas SQL son una herramienta poderosa para simplificar el trabajo del agente. Una vista es esencialmente una query guardada que puede ser consultada como si fuera una tabla. Crearemos una vista `sales_summary` que pre-calcula estadísticas agregadas comunes. Esto tiene dos beneficios: primero, queries que necesitan estas agregaciones se vuelven mucho más simples (el agente puede simplemente SELECT FROM la vista); segundo, para el LLM es más fácil razonar sobre una vista con nombre semántico que construir agregaciones complejas desde cero.

Nuestra vista agregará datos por modelo, región y año, calculando métricas como volumen total de ventas, precio promedio, kilometraje promedio y distribución de tipos de combustible. Estas son precisamente las dimensiones sobre las que stakeholders de negocio típicamente quieren analizar rendimiento.

In [ ]:
# Definimos la vista SQL de resumen de ventas
create_view_sql = """
CREATE VIEW IF NOT EXISTS sales_summary AS
SELECT
    model,
    product_line,
    region,
    year,
    price_segment,
    fuel_type,
    COUNT(*) as transaction_count,
    SUM(sales_volume) as total_volume,
    AVG(price_usd) as avg_price,
    AVG(mileage_km) as avg_mileage,
    AVG(vehicle_age) as avg_vehicle_age,
    AVG(price_per_km) as avg_price_per_km,
    AVG(annual_depreciation) as avg_annual_depreciation,
    AVG(engine_size_l) as avg_engine_size
FROM sales_transactions
GROUP BY model, product_line, region, year, price_segment, fuel_type
;
"""

# Ejecutamos la creación de la vista
with engine.connect() as conn:
    conn.execute(text(create_view_sql))
    conn.commit()

print("✓ Vista 'sales_summary' creada exitosamente")

# Verificamos la vista consultándola
test_query = """
SELECT model, region, year, total_volume, avg_price
FROM sales_summary
WHERE year >= 2020
ORDER BY total_volume DESC
LIMIT 10
"""
df_view_test = pd.read_sql_query(test_query, engine)
print("\n--- Top 10 Combinaciones Modelo-Región por Volumen (2020+) ---")
display(df_view_test)

### Verificación de Integridad del Esquema

Antes de entregar nuestra base de datos al agente SQL, ejecutamos una serie de tests de integridad para confirmar que el esquema funciona correctamente y que no hay inconsistencias en los datos. Esta verificación proactiva previene que el agente genere queries sobre estructuras defectuosas que producirían resultados incorrectos o errores crípticos.

In [ ]:
print("=== VERIFICACIÓN DE INTEGRIDAD DEL ESQUEMA ===")

# Test 1: Conteo de registros en cada tabla
with engine.connect() as conn:
    tables = ['sales_transactions', 'models', 'regions']
    for table in tables:
        result = conn.execute(text(f"SELECT COUNT(*) as count FROM {table}")).fetchone()
        print(f"  Tabla '{table}': {result[0]:,} registros")

# Test 2: Verificar que todos los modelos en transactions existen en catálogo
query_model_check = """
SELECT COUNT(DISTINCT st.model) as models_in_transactions,
       (SELECT COUNT(*) FROM models) as models_in_catalog
FROM sales_transactions st
"""
result = pd.read_sql_query(query_model_check, engine)
print(f"\n  Modelos en transacciones: {result['models_in_transactions'].values[0]}")
print(f"  Modelos en catálogo: {result['models_in_catalog'].values[0]}")

# Test 3: Verificar distribución de price_segment
query_segment_dist = """
SELECT price_segment, COUNT(*) as count
FROM sales_transactions
GROUP BY price_segment
ORDER BY
    CASE price_segment
        WHEN 'Budget' THEN 1
        WHEN 'Mid-range' THEN 2
        WHEN 'Premium' THEN 3
        WHEN 'Luxury' THEN 4
    END
"""
segment_dist = pd.read_sql_query(query_segment_dist, engine)
print("\n--- Distribución de Segmentos de Precio ---")
display(segment_dist)

# Test 4: Verificar integridad de vista sales_summary
query_view_check = "SELECT COUNT(*) as rows FROM sales_summary"
view_rows = pd.read_sql_query(query_view_check, engine)
print(f"\n  Filas en vista sales_summary: {view_rows['rows'].values[0]:,}")

print("\n✓ Verificaciones de integridad completadas. La base de datos está lista para consultas.")

## 6. Configuración del Agente SQL con LangChain

Llegamos al momento culminante: conectar nuestra base de datos cuidadosamente diseñada con un agente de lenguaje natural que puede razonar sobre ella. El agente SQL de LangChain es una pieza sofisticada de ingeniería que combina varias capacidades críticas. Primero, inspecciona automáticamente el esquema de la base de datos para entender qué tablas, columnas y relaciones están disponibles. Segundo, usa el modelo de lenguaje para interpretar preguntas en lenguaje natural y razonar sobre qué información necesita. Tercero, genera queries SQL sintácticamente correctas y semánticamente apropiadas. Cuarto, ejecuta esas queries y obtiene resultados. Finalmente, interpreta los resultados y formula respuestas en lenguaje natural que son comprensibles para usuarios no técnicos.

Este pipeline completo es invisible para el usuario final, quien simplemente hace preguntas como si estuviera hablando con un analista de datos humano experto. La magia está en que el agente no está pre-programado con queries específicas, sino que genera dinámicamente el SQL apropiado para cada pregunta única. Esta flexibilidad es lo que diferencia un sistema de reportes tradicional (respuestas predefinidas a preguntas predefinidas) de un asistente de inteligencia de negocio verdaderamente conversacional.

La configuración del agente requiere dos componentes principales: el objeto `SQLDatabase` de LangChain que actúa como interfaz con nuestra base SQLite, y el `ChatOpenAI` que proporciona las capacidades de razonamiento del modelo de lenguaje. Un parámetro crítico es `temperature=0`, que configura el modelo para que sea completamente determinista. Para generación de código SQL, no queremos creatividad o variación aleatoria; queremos precisión y consistencia. Cada vez que se hace la misma pregunta, deberíamos obtener la misma query SQL.

In [ ]:
import os
import sys

# Forzar codificación UTF-8 en todo el entorno
os.environ['PYTHONIOENCODING'] = 'utf-8'
os.environ['LANG'] = 'C.UTF-8'

# Para Colab específicamente
if 'google.colab' in sys.modules:
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

# Conectamos LangChain a nuestra base de datos
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print("Base de datos conectada. Tablas disponibles:")
print(db.get_usable_table_names())

print("\n--- Schema de la tabla principal ---")
print(db.get_table_info(['sales_transactions']))

# Recreamos el LLM con la API key correcta
llm = ChatOpenAI(
    model="gpt-5.1",

)

# Recreamos el agente
agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type="openai-tools",
    verbose=True,
    handle_parsing_errors=True
)

print("✓ Agente SQL configurado exitosamente")

## 7. Demostración: Preguntas de Negocio en Lenguaje Natural

Es hora de poner a prueba nuestro sistema con una serie de preguntas de negocio de complejidad creciente. Estas preguntas están diseñadas para demostrar diferentes capacidades del agente y, crucialmente, para evidenciar cómo el diseño de nuestra base de datos facilita o dificulta ciertos tipos de análisis.

Observa cuidadosamente el output verbose de cada query. Verás el proceso de razonamiento completo del agente: cómo examina el esquema, formula una estrategia, genera SQL (que a veces necesita iterar si hay errores de sintaxis o lógica), ejecuta la query, obtiene resultados y finalmente formula una respuesta en lenguaje natural. Esta transparencia es invaluable para debugging y para entender cómo decisiones de diseño del esquema impactan la eficiencia del agente.

### Pregunta 1: Análisis Básico de Agregación por Región

In [ ]:
query1 = "Cual es el volumen total de ventas por region"  # Sin tildes

print(f"PREGUNTA: {query1}\n")
print("="*80)
response1 = agent_executor.invoke({"input": query1})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response1['output']}")

### Pregunta 2: Uso de Features Calculadas - Segmentación de Mercado

Esta pregunta demuestra el valor del feature engineering. Sin la columna `price_segment` que creamos, el agente tendría que generar CASE statements complejos con rangos numéricos. Con la feature pre-calculada, la query es simple y semánticamente clara.

In [ ]:
query2 = "¿Cuál es la distribución de ventas por segmento de precio (Budget, Mid-range, Premium, Luxury)?"

print(f"PREGUNTA: {query2}\n")
print("="*80)
response2 = agent_executor.invoke({"input": query2})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response2['output']}")

### Pregunta 3: Análisis de Tendencias Temporales

Esta pregunta requiere agrupación temporal y cálculo de métricas agregadas. Demuestra cómo el agente puede razonar sobre evolución de mercado a lo largo del tiempo.

In [ ]:
query3 = "¿Cómo ha evolucionado el precio promedio de  los modelos de vehículos la base de datos (todos son de la marca BMW) desglosalo por modelo entre 2020 y 2024?"

print(f"PREGUNTA: {query3}\n")
print("="*80)
response3 = agent_executor.invoke({"input": query3})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response3['output']}")

### Pregunta 4: Uso de Jerarquía de Líneas de Producto

Esta pregunta aprovecha la columna `product_line` que categorizó modelos en SUV, Sedan, Performance, etc. Sin esta categorización, el agente tendría dificultades para entender qué constituye un "SUV" versus otros tipos de vehículos.

In [ ]:
query4 = "Compara el volumen de ventas entre SUVs y sedanes. ¿Qué línea de producto tiene mejor rendimiento?"

print(f"PREGUNTA: {query4}\n")
print("="*80)
response4 = agent_executor.invoke({"input": query4})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response4['output']}")

### Pregunta 5: Análisis de Adopción de Tecnologías Limpias

Esta pregunta sobre vehículos eléctricos y tipos de combustible es relevante para estrategia de sostenibilidad corporativa. Demuestra cómo el agente puede filtrar por características específicas y calcular proporciones.

In [ ]:
query5 = "¿Cuál es el porcentaje de vehículos eléctricos en el total de ventas? ¿Y cómo se distribuyen los tipos de combustible?"

print(f"PREGUNTA: {query5}\n")
print("="*80)
response5 = agent_executor.invoke({"input": query5})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response5['output']}")

### Pregunta 6: Análisis Complejo Multi-Dimensional

Esta pregunta requiere filtrado por múltiples dimensiones (región, tipo de combustible, segmento de precio) y cálculo de métricas agregadas. Es el tipo de análisis que tradicionalmente requeriría múltiples queries iterativas o una query SQL muy compleja escrita por un experto.

In [ ]:
query6 = "En Europa, ¿cuáles son los 5 modelos más vendidos de vehículos eléctricos en el segmento Premium?"

print(f"PREGUNTA: {query6}\n")
print("="*80)
response6 = agent_executor.invoke({"input": query6})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response6['output']}")

### Pregunta 7: Uso de Métricas Derivadas - Análisis de Valor

Esta pregunta usa la métrica `price_per_km` que calculamos en el feature engineering. Demuestra cómo features numéricas permiten análisis sofisticados de valor relativo.

In [ ]:
query7 = "¿Cuáles son los modelos con mejor relación precio-kilometraje (price_per_km más bajo) en el segmento Luxury?"

print(f"PREGUNTA: {query7}\n")
print("="*80)
response7 = agent_executor.invoke({"input": query7})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response7['output']}")

### Pregunta 8: Uso de la Vista sales_summary

Esta pregunta aprovecha la vista `sales_summary` que pre-calcula agregaciones. El agente puede consultar esta vista directamente en lugar de tener que construir agregaciones complejas desde la tabla base.

In [ ]:
query8 = "Usando la vista sales_summary, ¿cuál fue el modelo con mayor volumen total de ventas en 2023?"

print(f"PREGUNTA: {query8}\n")
print("="*80)
response8 = agent_executor.invoke({"input": query8})
print("="*80)
print(f"\n📊 RESPUESTA FINAL:\n{response8['output']}")

## 8. Análisis Reflexivo: Lecciones de Diseño para Sistemas de IA

Ahora que hemos visto el agente en acción respondiendo diversas preguntas de negocio, es momento de reflexionar críticamente sobre qué aspectos de nuestro diseño funcionaron bien y dónde hay oportunidades de mejora. Esta metacognición es lo que distingue entre simplemente "hacer que funcione" y desarrollar intuición profunda sobre diseño de sistemas de IA.

### Decisiones de Diseño que Amplificaron las Capacidades del Agente

**Feature engineering explícito versus cálculos dinámicos:** Al pre-calcular features como `price_segment`, `product_line` y `vehicle_age`, permitimos que el agente genere queries simples y legibles. Sin estas features, el agente tendría que generar expresiones CASE complejas o aritmética en cada query, lo cual es propenso a errores y más difícil de optimizar para el motor de base de datos.

**Normalización de nombres a snake_case:** Esta decisión aparentemente cosmética tiene impacto real. Los nombres consistentes y sin case sensitivity reducen errores de sintaxis en el SQL generado. El agente no tiene que recordar si una columna es `Fuel_Type` o `fuel_type` o `fuelType`; todas siguen la misma convención.

**Vista sales_summary como abstracción:** Esta vista pre-calcula agregaciones comunes, transformando queries potencialmente complejas en simples SELECTs. Para el LLM, es más fácil razonar sobre "consulta la vista sales_summary" que construir GROUP BYs anidados con múltiples agregaciones.

**Métricas de valor relativo:** Columnas como `price_per_km` y `annual_depreciation` permiten comparaciones sofisticadas usando operadores SQL estándar. Sin estas métricas, preguntas sobre "mejor valor" o "depreciación rápida" serían ambiguas o imposibles de responder con SQL puro.

### Limitaciones y Oportunidades de Mejora

**Ausencia de índices optimizados:** Para un dataset de 50,000+ registros, columnas frecuentemente filtradas como `region`, `year`, `model` y `price_segment` deberían tener índices. Sin índices, queries con filtros complejos pueden ser lentas. SQLite crea algunos índices automáticamente, pero no de forma óptima.

**Falta de metadata temporal granular:** El dataset contiene años pero no fechas específicas de venta. Esto limita análisis de estacionalidad o tendencias mensuales. En un sistema de producción, tendríamos timestamps completos.

**Jerarquía de regiones no desarrollada:** Actualmente `region` es un string plano. En producción, tendríamos una jerarquía (región -> país -> continente) que permitiría agregaciones multi-nivel como "analiza Europa como conjunto" o "compara rendimiento de países dentro de Asia".

**Sin modelado de clientes o dealers:** Este dataset representa transacciones anónimas. Un sistema real tendría información sobre quién compró (segmento de cliente) y dónde (dealer específico), permitiendo análisis mucho más ricos sobre comportamiento de compra y rendimiento de canales.

### Limitaciones Fundamentales de Agentes SQL

Incluso con el mejor diseño de base de datos posible, hay ciertos tipos de preguntas que los agentes SQL no pueden responder bien:

**Análisis causal:** SQL puede decir "qué pasó" pero no "por qué pasó". Preguntas como "¿por qué cayeron las ventas de sedanes en 2022?" requieren razonamiento causal que va más allá de correlaciones estadísticas que SQL puede calcular.

**Forecasting:** Preguntas sobre el futuro ("¿cuáles serán las ventas de vehículos eléctricos en 2025?") requieren modelos predictivos que no son parte de SQL estándar. Necesitarías integrar con librerías de machine learning.

**Análisis de texto no estructurado:** Si tuviéramos campos de texto libre como "comentarios de clientes" o "notas de ventas", SQL no puede hacer análisis semántico profundo. Para eso necesitarías técnicas de NLP o embeddings vectoriales.

**Comparaciones cross-dataset:** Si quisieras comparar ventas de BMW con datos de competidores que viven en otra base de datos o formato, el agente SQL tendría dificultades. Necesitarías un orquestador de nivel superior.

Entender estas limitaciones es tan importante como entender las capacidades. El diseño efectivo de sistemas de IA requiere saber cuándo SQL es la herramienta correcta y cuándo necesitas aproximaciones complementarias.

## 9. Conclusiones y Extensiones Avanzadas

Has completado un ejercicio completo que simula un proyecto real de ingeniería de IA aplicada a inteligencia de negocio. Transformaste datos crudos de ventas de vehículos en un sistema conversacional que permite a stakeholders no técnicos obtener insights sofisticados mediante preguntas en lenguaje natural. Este tipo de sistema es exactamente lo que empresas de todos los tamaños necesitan para democratizar el acceso a datos y acelerar la toma de decisiones informadas.

### Competencias Desarrolladas

Has dominado el pipeline completo de preprocesamiento de datos orientado a IA, desde identificar oportunidades de feature engineering hasta implementar transformaciones que amplifican capacidades de razonamiento. Has aprendido a diseñar esquemas relacionales con empatía hacia agentes LLM, considerando explícitamente qué queries serán comunes y cómo facilitarlas mediante estructuras apropiadas. Has configurado y probado un agente SQL con LangChain, entendiendo tanto sus capacidades como sus limitaciones. Y has desarrollado intuición sobre el balance entre normalización, desnormalización y agregación pre-calculada.

### Extensiones Avanzadas para Profundizar

Si quieres llevar este proyecto más allá como ejercicio de aprendizaje o como base para un proyecto profesional, considera estas direcciones:

**Integración con embeddings vectoriales para búsqueda híbrida:** Agrega una columna con embeddings de descripciones de modelos (generados con text-embedding-3-small de OpenAI). Esto permitiría queries semánticas como "encuentra modelos similares al X5 en características y posicionamiento" que van más allá de lo que SQL estructural puede hacer.

**Dashboard interactivo con Streamlit:** Conecta la base de datos a un dashboard web usando Streamlit o Plotly Dash. El agente SQL podría alimentar visualizaciones dinámicas que se actualizan basándose en preguntas del usuario, combinando la potencia de SQL con la claridad de gráficos.

**Sistema de alertas automáticas:** Implementa queries programadas que detecten anomalías (por ejemplo, "ventas de un modelo cayeron más de 20% mes a mes") y generen alertas automáticas. El agente podría incluso generar explicaciones textuales de qué cambió.

**Modelo de predicción de precios:** Entrena un modelo de machine learning (Random Forest o XGBoost) para predecir precio de vehículos basándose en características. Integra este modelo con el agente SQL para responder preguntas como "¿qué precio debería tener un X3 del 2023 con 50,000 km?"

**Análisis competitivo con múltiples marcas:** Extiende el dataset para incluir ventas de Mercedes, Audi y otros competidores. Modifica el esquema para soportar comparaciones cross-brand y análisis de market share.

**Pipeline de actualización automatizada:** Implementa un sistema que ingiere nuevos datos de ventas periódicamente, los procesa automáticamente a través de tu pipeline de feature engineering, y actualiza la base de datos sin intervención manual.

**Sistema de permisos y seguridad:** En producción, diferentes usuarios deberían tener acceso a diferentes niveles de detalle. Implementa Row-Level Security (RLS) para filtrar automáticamente datos según el rol del usuario.

### Reflexión Final: Diseño Centrado en IA como Competencia Emergente

Lo que has aprendido aquí trasciende esta implementación específica. Has desarrollado una forma de pensar sobre estructuración de datos que considera explícitamente las capacidades y limitaciones de sistemas de IA. Este "diseño centrado en IA" es una competencia emergente cada vez más valiosa en la industria.

No basta con organizar datos de forma lógicamente correcta según principios tradicionales de normalización de bases de datos. Debemos diseñar infraestructuras de datos pensando en cómo serán consumidas por agentes de IA, qué abstracciones facilitan su razonamiento, qué features amplífican sus capacidades, y qué estructuras minimizan la complejidad de las tareas que deben realizar.

A medida que los sistemas de IA se vuelven más capaces y omnipresentes, esta habilidad de diseñar datos "AI-ready" se convertirá en una ventaja competitiva fundamental. Has dado un paso sólido en esa dirección.

---

**Fin del Ejercicio Práctico - Pipeline de Datos BMW con Agente SQL**